In [59]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [60]:
spark=SparkSession.builder.appName('stackoverflow').getOrCreate()

In [72]:
so_df = spark.read.load("/Users/sae/Downloads/stack-overflow-developer-survey-2019/survey_results_public.csv",format = "csv" , header=True, inferschema=True)

In [73]:
print(str(so_df.printSchema()))

root
 |-- Respondent: integer (nullable = true)
 |-- MainBranch: string (nullable = true)
 |-- Hobbyist: string (nullable = true)
 |-- OpenSourcer: string (nullable = true)
 |-- OpenSource: string (nullable = true)
 |-- Employment: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Student: string (nullable = true)
 |-- EdLevel: string (nullable = true)
 |-- UndergradMajor: string (nullable = true)
 |-- EduOther: string (nullable = true)
 |-- OrgSize: string (nullable = true)
 |-- DevType: string (nullable = true)
 |-- YearsCode: string (nullable = true)
 |-- Age1stCode: string (nullable = true)
 |-- YearsCodePro: string (nullable = true)
 |-- CareerSat: string (nullable = true)
 |-- JobSat: string (nullable = true)
 |-- MgrIdiot: string (nullable = true)
 |-- MgrMoney: string (nullable = true)
 |-- MgrWant: string (nullable = true)
 |-- JobSeek: string (nullable = true)
 |-- LastHireDate: string (nullable = true)
 |-- LastInt: string (nullable = true)
 |-- FizzBuzz: 

In [74]:
print(str(so_df.columns))

['Respondent', 'MainBranch', 'Hobbyist', 'OpenSourcer', 'OpenSource', 'Employment', 'Country', 'Student', 'EdLevel', 'UndergradMajor', 'EduOther', 'OrgSize', 'DevType', 'YearsCode', 'Age1stCode', 'YearsCodePro', 'CareerSat', 'JobSat', 'MgrIdiot', 'MgrMoney', 'MgrWant', 'JobSeek', 'LastHireDate', 'LastInt', 'FizzBuzz', 'JobFactors', 'ResumeUpdate', 'CurrencySymbol', 'CurrencyDesc', 'CompTotal', 'CompFreq', 'ConvertedComp', 'WorkWeekHrs', 'WorkPlan', 'WorkChallenge', 'WorkRemote', 'WorkLoc', 'ImpSyn', 'CodeRev', 'CodeRevHrs', 'UnitTests', 'PurchaseHow', 'PurchaseWhat', 'LanguageWorkedWith', 'LanguageDesireNextYear', 'DatabaseWorkedWith', 'DatabaseDesireNextYear', 'PlatformWorkedWith', 'PlatformDesireNextYear', 'WebFrameWorkedWith', 'WebFrameDesireNextYear', 'MiscTechWorkedWith', 'MiscTechDesireNextYear', 'DevEnviron', 'OpSys', 'Containers', 'BlockchainOrg', 'BlockchainIs', 'BetterLife', 'ITperson', 'OffOn', 'SocialMedia', 'Extraversion', 'ScreenName', 'SOVisit1st', 'SOVisitFreq', 'SOVisi

In [76]:
so_df = so_df.withColumn("Age", so_df.Age.cast('integer'))
so_df.select("Gender","Age").groupBy("Gender").avg().show()

+--------------------+------------------+
|              Gender|          avg(Age)|
+--------------------+------------------+
|                 Man|30.424181874074282|
|                  NA| 30.35640785781104|
|Man;Non-binary, g...|28.908536585365855|
|           Woman;Man| 26.23170731707317|
|Woman;Man;Non-bin...| 29.78723404255319|
|Woman;Non-binary,...|28.210884353741495|
|Non-binary, gende...|29.018036072144287|
|               Woman| 29.49538679914833|
+--------------------+------------------+



In [90]:
so_df.select("Country", "Respondent").groupBy("Country").count().sort(desc("count")).show(5)

+--------------+-----+
|       Country|count|
+--------------+-----+
| United States|20949|
|         India| 9061|
|       Germany| 5866|
|United Kingdom| 5737|
|        Canada| 3395|
+--------------+-----+
only showing top 5 rows



In [91]:
so_df.select("Country", "Respondent") \
     .groupby("Country") \
     .agg(count("Respondent"). alias("Respondent_countrywise")) \
     .sort(desc("Respondent_countrywise")).show(5)

+--------------+----------------------+
|       Country|Respondent_countrywise|
+--------------+----------------------+
| United States|                 20949|
|         India|                  9061|
|       Germany|                  5866|
|United Kingdom|                  5737|
|        Canada|                  3395|
+--------------+----------------------+
only showing top 5 rows



In [120]:
so_df = so_df.withColumn("Age1stCode", when(so_df.Age1stCode == 'Younger than 5 years', 4)\
                .when(so_df.Age1stCode == "Older than 85", 86)\
                .when(so_df.Age1stCode == "NA", None).otherwise(so_df.Age1stCode))

In [123]:
so_df = so_df.withColumn("Age1stCode", so_df.Age1stCode.cast("integer"))
so_df.select("Age1stCode").sort(desc("Age1stCode")).show(3)

+----------+
|Age1stCode|
+----------+
|        86|
|        86|
|        86|
+----------+
only showing top 3 rows



In [ ]:
so_df.select

In [128]:
mask_country = (so_df.Country=='United States') 
mask_major1 = (so_df.UndergradMajor=='Mathematics or statistics')
mask_major2 = (so_df.UndergradMajor=='Computer science, computer engineering, or software engineering')

so_df.select("Country", "Age1stCode") \
     .filter(mask_country & (mask_major1 | mask_major2)) \
     .groupBy("Country") \
     .max("Age1stCode") \
     .show()

+-------------+---------------+
|      Country|max(Age1stCode)|
+-------------+---------------+
|United States|             56|
+-------------+---------------+



In [129]:
spark.stop()